In [1]:
%pip install fiftyone
%pip install torch torchvision
%pip install pycocotools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import fiftyone as fo
import csv
import torch

In [3]:
!git clone https://github.com/pytorch/vision.git

fatal: destination path 'vision' already exists and is not an empty directory.


In [4]:
%cd vision

!git checkout v0.3.0

!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../

/home/jupyter-kphan3/vision
HEAD is now at be376084d8 version check against PyTorch's CUDA version


In [5]:
%cd ..

/home/jupyter-kphan3


In [2]:
%ls

coco_eval.py   oidv7-class-descriptions-boxable-modified.csv  utils.py
coco_utils.py  __pycache__/                                   vision/
engine.py      transforms.py
fiftyone/      Untitled.ipynb


In [18]:
classes = []

#read in targeted classes
with open('oidv7-class-descriptions-boxable-modified.csv') as classes_file:
    reader_obj = csv.reader(classes_file)
    for index, line in enumerate(reader_obj):
      if index == 0:
        continue

      classes.append(line[1])

dataset = fo.zoo.load_zoo_dataset("open-images-v7",
                                  split="train",
                                  label_types=["detections"],
                                  classes=classes,
                                  max_samples=40000,
                                  )


Found 5000 images, downloading the remaining 35000
 100% |███████████████| 35000/35000 [6.0m elapsed, 0s remaining, 53.6 files/s]       
Dataset info written to '/home/jupyter-kphan3/fiftyone/open-images-v7/info.json'
Loading 'open-images-v7' split 'train'
 100% |█████████████| 40000/40000 [2.5m elapsed, 0s remaining, 248.2 samples/s]      
Dataset 'open-images-v7-train-40000' created


In [19]:
test_dataset = fo.zoo.load_zoo_dataset("open-images-v7",
                                  split="test",
                                  label_types=["detections"],
                                  classes=classes,
                                  max_samples=12500,
                                  )

Found 1500 images, downloading the remaining 11000
 100% |███████████████| 11000/11000 [1.9m elapsed, 0s remaining, 68.1 files/s]       
Dataset info written to '/home/jupyter-kphan3/fiftyone/open-images-v7/info.json'
Loading 'open-images-v7' split 'test'
 100% |█████████████| 12500/12500 [47.0s elapsed, 0s remaining, 286.9 samples/s]      
Dataset 'open-images-v7-test-12500' created


In [20]:
dataset.compute_metadata()
dataset.exists('metadata', False)

Computing metadata...
 100% |█████████████| 40000/40000 [1.3m elapsed, 0s remaining, 895.7 samples/s]      


Dataset:     open-images-v7-train-40000
Media type:  image
Num samples: 0
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Exists(field='metadata', bool=False)

In [21]:
test_dataset.compute_metadata()
test_dataset.exists('metadata', False)

Computing metadata...
 100% |█████████████| 12500/12500 [27.8s elapsed, 0s remaining, 512.8 samples/s]      


Dataset:     open-images-v7-test-12500
Media type:  image
Num samples: 0
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Exists(field='metadata', bool=False)

In [22]:
# load Faster-RCNN w/ ResNet50 backbone from Torchvision and modify the classifer for the # of classes we are training on
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
  weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features

  # replace pretrained head w/ new head
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  return model

In [23]:
import torch
import fiftyone.utils.coco as fouc
from PIL import Image

class FiftyOneTorchDataset(torch.utils.data.Dataset):
  # construct a PyTorch dataset from a FiftyOne dataset

  def __init__(self, fiftyone_dataset, transforms=None, gt_field="ground_truth", classes=None):
    self.samples = fiftyone_dataset
    self.transforms = transforms
    self.gt_field = gt_field
    self.img_paths = self.samples.values("filepath")
    self.classes = classes

    if not self.classes:
      self.classes = self.samples.distinct("%s.detections.label" % gt_field)

    if self.classes[0] != "background":
        self.classes = ["background"] + self.classes

    self.labels_map_rev = {c: i for i,c in enumerate(self.classes)}


  def __getitem__(self, idx):
    img_path = self.img_paths[idx]
    sample = self.samples[img_path]
    metadata = sample.metadata

    img = Image.open(img_path).convert("RGB")

    boxes = []
    labels = []
    area = []
    iscrowd = []

    detections = sample[self.gt_field].detections
    for det in detections:
      category_id = self.labels_map_rev[det.label]
      coco_obj = fouc.COCOObject.from_label(det, metadata, category_id=category_id)

      x,y,w,h = coco_obj.bbox
      boxes.append([x,y,x+w,y+h])
      labels.append(coco_obj.category_id)
      area.append(coco_obj.area)
      iscrowd.append(coco_obj.iscrowd)

    target = {}
    target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
    target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
    target["image_id"] = torch.as_tensor([idx])
    target["area"] = torch.as_tensor(area, dtype=torch.float32)
    target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

    if self.transforms is not None:
      img, target = self.transforms(img, target)

    return img, target

  def __len__(self):
    return len(self.img_paths)

  def get_classes(self):
    return self.classes


In [24]:
import transforms as T

# freeze layers in the backbone and define data transformations
    
train_transforms = T.Compose([T.ToTensor(), T.RandomHorizontalFlip(0.5)])
test_transforms = T.Compose([T.ToTensor()])

In [25]:
# get the fifty_one views, pass those views into the PyTorch dataset converter
from fiftyone import ViewField as F

train_view = dataset.filter_labels("ground_truth", F("label").is_in(classes))

test_view = test_dataset.filter_labels("ground_truth", F("label").is_in(classes))

# validation_view = validation_dataset.filter_labels("ground_truth", F("label").is_in(classes))

torch_dataset_train = FiftyOneTorchDataset(train_view, train_transforms, classes=classes)
torch_dataset_test = FiftyOneTorchDataset(test_view, test_transforms, classes=classes)

In [26]:
num_classes = len(torch_dataset_train.get_classes())

model = get_model(num_classes+1)

In [27]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [28]:
from engine import train_one_epoch, evaluate
import utils

def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4):
    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=5, shuffle=True, num_workers=0,
        collate_fn=utils.collate_fn)

    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=3,
                                                    gamma=0.1)

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

        # update the learning rate
        lr_scheduler.step()
#         # evaluate on the test dataset
#         evaluate(model, data_loader_test, device=device)
        

In [29]:
do_training(model, torch_dataset_train, torch_dataset_test, num_epochs=4)
torch.save(model.state_dict(), 'model.pth')

Using device cuda
Epoch: [0]  [   0/8000]  eta: 1:41:51  lr: 0.000010  loss: 4.6599 (4.6599)  loss_classifier: 4.5008 (4.5008)  loss_box_reg: 0.1404 (0.1404)  loss_objectness: 0.0148 (0.0148)  loss_rpn_box_reg: 0.0039 (0.0039)  time: 0.7640  data: 0.0658  max mem: 11886
Epoch: [0]  [  10/8000]  eta: 1:17:09  lr: 0.000060  loss: 4.6765 (4.7415)  loss_classifier: 4.4845 (4.4675)  loss_box_reg: 0.2258 (0.2341)  loss_objectness: 0.0202 (0.0255)  loss_rpn_box_reg: 0.0077 (0.0144)  time: 0.5795  data: 0.0502  max mem: 14090
Epoch: [0]  [  20/8000]  eta: 1:15:20  lr: 0.000110  loss: 4.5919 (4.5437)  loss_classifier: 4.3093 (4.2346)  loss_box_reg: 0.2302 (0.2561)  loss_objectness: 0.0232 (0.0345)  loss_rpn_box_reg: 0.0104 (0.0185)  time: 0.5566  data: 0.0496  max mem: 14090
Epoch: [0]  [  30/8000]  eta: 1:15:48  lr: 0.000160  loss: 3.8368 (4.1227)  loss_classifier: 3.4178 (3.7970)  loss_box_reg: 0.2552 (0.2661)  loss_objectness: 0.0278 (0.0400)  loss_rpn_box_reg: 0.0172 (0.0196)  time: 0.5659 

In [30]:
import fiftyone as fo

def convert_torch_predictions(preds, det_id, s_id, w, h, classes):
    # Convert the outputs of the torch model into a FiftyOne Detections object
    dets = []
    for bbox, label, score in zip(
        preds["boxes"].cpu().detach().numpy(),
        preds["labels"].cpu().detach().numpy(),
        preds["scores"].cpu().detach().numpy()
    ):
        # Parse prediction into FiftyOne Detection object
        x0,y0,x1,y1 = bbox
        coco_obj = fouc.COCOObject(det_id, s_id, int(label), [x0, y0, x1-x0, y1-y0])
        det = coco_obj.to_detection((w,h), classes)
        det["confidence"] = float(score)
        dets.append(det)
        det_id += 1

    detections = fo.Detections(detections=dets)

    return detections, det_id

def add_detections(model, torch_dataset, view, field_name="predictions"):
    # Run inference on a dataset and add results to FiftyOne
    torch.set_num_threads(1)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    model.eval()
    model.to(device)
    image_paths = torch_dataset.img_paths
    classes = torch_dataset.classes
    det_id = 0

    with fo.ProgressBar() as pb:
        for img, targets in pb(torch_dataset):
            # Get FiftyOne sample indexed by unique image filepath
            img_id = int(targets["image_id"][0])
            img_path = image_paths[img_id]
            sample = view[img_path]
            s_id = sample.id
            w = sample.metadata["width"]
            h = sample.metadata["height"]

            # Inference
            preds = model(img.unsqueeze(0).to(device))[0]

            detections, det_id = convert_torch_predictions(
                preds,
                det_id,
                s_id,
                w,
                h,
                classes,
            )

            sample[field_name] = detections
            sample.save()

In [31]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device %s" % device)
model.load_state_dict(torch.load('./model.pth'))
model.to(device)

add_detections(model, torch_dataset_test, test_dataset, field_name="predictions")

Using device cuda
Using device cuda
 100% |█████████████| 12500/12500 [15.0m elapsed, 0s remaining, 14.3 samples/s]      


In [32]:
results = fo.evaluate_detections(
    test_view,
    "predictions",
    classes=classes,
    eval_key="eval",
    compute_mAP=True
)

Evaluating detections...
 100% |█████████████| 12500/12500 [14.0m elapsed, 0s remaining, 21.1 samples/s]      
Performing IoU sweep...
 100% |█████████████| 12500/12500 [4.4m elapsed, 0s remaining, 66.2 samples/s]      


In [33]:
results.mAP()

0.2202485238089921

In [34]:
results.print_report()

                         precision    recall  f1-score   support

                  Apple       0.05      0.67      0.09       353
              Artichoke       0.28      0.36      0.32        66
                  Bagel       0.12      0.73      0.20       197
            Baked goods       0.09      0.75      0.17      3046
                 Banana       0.03      0.67      0.06       135
                   Beer       0.07      0.93      0.13       285
            Bell pepper       0.07      0.76      0.12       190
                  Bread       0.11      0.66      0.19      1400
               Broccoli       0.06      0.89      0.12       171
                Burrito       0.00      0.00      0.00       121
                Cabbage       0.12      0.55      0.20       102
                   Cake       0.27      0.65      0.38       891
                  Candy       0.12      0.32      0.17       750
             Cantaloupe       0.00      0.00      0.00        26
                 Carrot 